In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os, json

In [ ]:
path_to_data = "/home/basti/Dropbox (UiO)/MEK3700/data/"

dataset = "brain2dclipp1"

# Read and show binary masks

In [ ]:
brainmask = np.load(path_to_data + dataset +  "/masks/mask.npy")
box = np.load(path_to_data + dataset +  "/masks/box.npy")

roi = brainmask * box

In [ ]:
plt.figure(dpi=200)
plt.imshow(brainmask)
plt.show()

plt.figure(dpi=200)
plt.imshow(roi)
plt.show()

plt.figure(dpi=200)
plt.imshow(brainmask + 2 * roi)
plt.show()

# Read images to dict

In [ ]:
def load_images(path_to_data, dataset):
    path_to_concentrations = path_to_data + dataset +  "/concentrations/"
    
    images = {}
    
    for cfile in os.listdir(path_to_concentrations):
        
        c = np.load(path_to_concentrations + cfile)
        
        images[cfile[:-4]] = c
        
    return images
    

In [ ]:
images = load_images(path_to_data, dataset)

In [ ]:
images.keys()

In [ ]:
plt.figure(dpi=200)
plt.imshow(images["24.00"] * brainmask)
plt.colorbar()
plt.show()

# Use json and dicts to store parameters

In [ ]:
param_dict ={"a": 1}
param_dict["b"] = 2

print("dict:", param_dict)
print("keys:", param_dict.keys())
print("items:", param_dict.items())
print("item a", param_dict["a"])

In [ ]:
exportfolder = "/home/basti/programming/pytorchdemo/"
with open(exportfolder + 'my_parameters.json', 'w') as fp:
    json.dump(param_dict, fp, sort_keys=True, indent=4)
    
with open(exportfolder + 'my_parameters.json', 'r') as data_file:    
    loaded_dict = json.load(data_file)
    
print(param_dict)
print("Loaded back:", loaded_dict)

# Create spatial coordinates

We first create a 256x256x2 array where arr[i,j, :] = (x_i, y_i) is the position of voxel (i,j)

In [ ]:
def make_coordinate_grid(images):
    """ Create a (n x n x 2) array where arr[i,j, :] = (x_i, y_i) is the position of voxel (i,j)"""
    n = 256

    # We want to assign coordinates to every voxel, so the shape of the meshgrid has to be the same as the image
    assert n == images[next(iter(images.keys()))].shape[0]
    assert n == images[next(iter(images.keys()))].shape[1]
    
    coordinate_axis = np.linspace(-0.5, 0.5, n)
    
    XX, YY = np.meshgrid(coordinate_axis, coordinate_axis, indexing='ij')
    
    arr = np.array([XX, YY])

    coordinate_grid = np.swapaxes(arr, 0, 1)
    coordinate_grid = np.swapaxes(coordinate_grid, 1, 2)
    
    return coordinate_grid

In [ ]:
coordinate_grid = make_coordinate_grid(images)

In [ ]:
plt.figure(dpi=200)
plt.plot(coordinate_grid[..., 0], coordinate_grid[..., 1], marker=".", linewidth=0, markersize=0.1, color="k")
# plt.xlim(0, 0.1)
plt.show()

# Map out voxels inside the brain, and get only their coordinates as a list



In [ ]:
def get_domain_coordinates(coordinate_grid, mask):
    return coordinate_grid[mask]   

In [ ]:
xy = get_domain_coordinates(coordinate_grid, mask=brainmask)

In [ ]:
def get_input_output_pairs(coordinate_grid, mask, images):
    
    input_output_pairs = {}
    
    xy = get_domain_coordinates(coordinate_grid, mask)
    
    for timekey, image in images.items():
        
        xyt = np.zeros((xy.shape[0], 3))
        xyt[..., :2] = xy
        xyt[..., -1] = float(timekey)
        
        input_output_pairs[timekey] = (xyt, image[mask])
        
    return input_output_pairs

In [ ]:
datadict = get_input_output_pairs(coordinate_grid, mask=brainmask, images=images)

In [ ]:
# t = "00.00"
t = "24.00"
xyt = datadict[t][0]
data_at_t = datadict[t][1]

### xyt is an array where xyt[i,j, :] = (x_i, y_j, t)  (voxel coordinates at first two indices and time at last index)

In [ ]:
xyt[1, :]

In [ ]:
data_at_t

In [ ]:
plt.figure(dpi=200)
plt.plot(xyt[..., 0], xyt[..., 1], marker=".", linewidth=0, markersize=0.1, color="k")
# plt.xlim(0, 0.1)
plt.xlabel("x", fontsize=12)
plt.ylabel("y", fontsize=12)
plt.show()

In [ ]:
plt.figure(dpi=200)
plt.scatter(xyt[..., 0], xyt[..., 1], c=data_at_t)
# plt.xlim(0, 0.1)
plt.colorbar()
plt.xlabel("x", fontsize=12)
plt.ylabel("y", fontsize=12)
plt.show()

# Do the same for the small region

In [ ]:
datadict = get_input_output_pairs(coordinate_grid, mask=roi, images=images)

In [ ]:
# t = "00.00"
t = "24.00"
xyt = datadict[t][0]
data_at_t = datadict[t][1]

In [ ]:
plt.figure(dpi=200)
plt.scatter(xyt[..., 0], xyt[..., 1], c=data_at_t)
# plt.xlim(0, 0.1)
plt.colorbar()
plt.xlabel("x", fontsize=12)
plt.ylabel("y", fontsize=12)
plt.show()